In [1]:
!pip install -U diffusers
!pip install flask pyngrok
print('all dependencies have been satisfied')

all dependencies have been satisfied


In [2]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode

# Load pipeline (low precision for speed)
pipe = DiffusionPipeline.from_pretrained(
    "cerspense/zeroscope_v2_576w",
    torch_dtype=torch.float16
)
pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after 

TextToVideoSDPipeline {
  "_class_name": "TextToVideoSDPipeline",
  "_diffusers_version": "0.34.0",
  "_name_or_path": "cerspense/zeroscope_v2_576w",
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet3DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
#@title 4️⃣ Generate video frames
from PIL import Image
import numpy as np
# Adjust num_inference_steps, num_frames, height & width as needed
output = pipe(
    prompt=prompt,
    num_inference_steps=50,
    num_frames=30,
    guidance_scale=7.5,
    height=320,
    width=576,
)
frames = output.frames

NameError: name 'prompt' is not defined

In [ ]:
from diffusers.utils import export_to_video
from PIL import Image
import numpy as np
from IPython.display import HTML
from base64 import b64encode

# Step 1: Convert to NumPy if needed
frames_np = output.frames
if not isinstance(frames_np, np.ndarray):
    frames_np = frames_np.cpu().numpy()

# Step 2: Normalize and convert to uint8
frames_np = (frames_np * 255).clip(0, 255).astype(np.uint8)

# Step 3: Remove batch dim -> shape: (8, 256, 256, 3)
frames_np = frames_np[0]

# Step 4: Convert each frame to a PIL Image
frames = [Image.fromarray(frame) for frame in frames_np]

# Step 5: Export to video
video_path = export_to_video(frames, output_video_path="mvp_output.mp4")

# Step 6: Show in notebook
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=500 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
from flask import Flask, request, jsonify

from pyngrok import ngrok
ngrok.set_auth_token("2wtPJIWVn0SinCkBiMqqqLkQpTc_3RVbVD7KUkQKHURdW4t1a")

In [ ]:
from flask import Flask, request, send_file
from flask_ngrok import run_with_ngrok
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video
from PIL import Image
import numpy as np
import torch

app = Flask(__name__)

# Load the model
pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.to("cuda")

@app.route("/generate-video", methods=["POST"])
def generate_video():
    req_data = request.get_json(force=True)  # ensures it parses even if no proper headers
    if not req_data or "prompt" not in req_data:
        return {"error": "No prompt provided"}, 400

    prompt = req_data["prompt"]

    output = pipe(
                prompt=prompt,
                num_inference_steps=50,
                num_frames=30,
                guidance_scale=7.5,
                height=320,
                width=576,
            )

    # Convert frames to uint8
    frames_np = output.frames
    if not isinstance(frames_np, np.ndarray):
        frames_np = frames_np.cpu().numpy()
    frames_np = (frames_np * 255).clip(0, 255).astype(np.uint8)
    frames_np = frames_np[0]
    frames = [Image.fromarray(f) for f in frames_np]

    # Export to video
    video_path = export_to_video(frames, output_video_path="video.mp4")

    return send_file("video.mp4", mimetype="video/mp4")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after 

In [ ]:
public_url = ngrok.connect(5000)
print("Public URL:", public_url)
app.run(port=5000)

Public URL: NgrokTunnel: "https://b16bf097eee0.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [13/Jul/2025 07:31:09] "POST /generate-video HTTP/1.1" 200 -
